In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

# Path to the chromedriver
chrome_driver_path = r"C:\Users\New\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
service = Service(executable_path=chrome_driver_path)

def amazon_login(driver, email, password):
    try:
        driver.get("https://www.amazon.in/ap/signin")
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "ap_email"))).send_keys(email)
        driver.find_element(By.ID, "continue").click()
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "ap_password"))).send_keys(password)
        driver.find_element(By.ID, "signInSubmit").click()
        print("Login successful.")
    except Exception as e:
        print("Login failed:", e)
        driver.quit()
        exit()

def scrape_product_reviews(driver, product_url):
    driver.get(product_url)
    time.sleep(30)  # Allow the page to load

    reviews_data = []
    try:
        # Navigate to reviews tab
        reviews_tab = driver.find_element(By.XPATH, "//a[@data-hook='see-all-reviews-link-foot']")
        reviews_tab.click()
        time.sleep(3)

        while True:
            reviews = driver.find_elements(By.XPATH, "//li[@data-hook='review']")
            for review in reviews:
                # Extract reviewer name
                try:
                    reviewer_name = review.find_element(By.XPATH, ".//span[@class='a-profile-name']").text
                except:
                    reviewer_name = "N/A"

                # Extract review date
                try:
                    review_date = review.find_element(By.XPATH, ".//span[@data-hook='review-date']").text
                except:
                    review_date = "N/A"

                # Extract review text
                try:
                    review_text = review.find_element(By.XPATH, ".//div[@class='a-row a-spacing-small review-data']").text
                except:
                    review_text = "N/A"

                # Extract review title
                try:
                    review_title = review.find_element(By.XPATH, ".//a[@data-hook='review-title']").text
                except:
                    review_title = "N/A"

                # Extract review rating
                try:
                    review_rating = review.find_element(By.XPATH, ".//span[@class='a-icon-alt']").text
                except:
                    review_rating = "N/A"

                reviews_data.append({
                    "Reviewer Name": reviewer_name,
                    "Review Title": review_title,
                    "Review Date": review_date,
                    "Review Text": review_text,
                    "Review Rating": review_rating,
                })

            # Check for next button and navigate
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "li.a-last a")
                next_button.click()
                time.sleep(3)
            except:
                break

    except Exception as e:
        print("Error while scraping reviews:", e)

    return reviews_data


def main():
    driver = webdriver.Chrome(service=service)

    # Log in to Amazon
    email = input("Enter your Amazon email: ")
    password = input("Enter your Amazon password: ")
    amazon_login(driver, email, password)

    product_url = input("Enter the URL of the product to scrape reviews: ")

    reviews = scrape_product_reviews(driver, product_url)

    # Save reviews to an Excel file
    df = pd.DataFrame(reviews)
    safe_product_name = re.sub(r'[\\/:*?"<>|]', '', product_url.split('/')[3])  # Extract product name from URL
    output_filename = f"{safe_product_name}_reviews.xlsx"
    df.to_excel(output_filename, index=False)

    print(f"Reviews saved to {output_filename}")

    driver.quit()

if __name__ == "__main__":
    main()


Enter your Amazon email:  9344267871
Enter your Amazon password:  Choco123$$


Login failed: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A3E56F15+28773]
	(No symbol) [0x00007FF7A3DC2600]
	(No symbol) [0x00007FF7A3C58FAA]
	(No symbol) [0x00007FF7A3CAF286]
	(No symbol) [0x00007FF7A3CAF4BC]
	(No symbol) [0x00007FF7A3D02A27]
	(No symbol) [0x00007FF7A3CD728F]
	(No symbol) [0x00007FF7A3CFF6F3]
	(No symbol) [0x00007FF7A3CD7023]
	(No symbol) [0x00007FF7A3C9FF5E]
	(No symbol) [0x00007FF7A3CA11E3]
	GetHandleVerifier [0x00007FF7A41A425D+3490733]
	GetHandleVerifier [0x00007FF7A41BBA43+3586963]
	GetHandleVerifier [0x00007FF7A41B147D+3544525]
	GetHandleVerifier [0x00007FF7A3F1C9DA+838442]
	(No symbol) [0x00007FF7A3DCD04F]
	(No symbol) [0x00007FF7A3DC9614]
	(No symbol) [0x00007FF7A3DC97B6]
	(No symbol) [0x00007FF7A3DB8CE9]
	BaseThreadInitThunk [0x00007FFFC944E8D7+23]
	RtlUserThreadStart [0x00007FFFC9FFBF2C+44]



In [ ]:
https://www.amazon.in/gp/product/B0BGQMGNCW/ref=ox_sc_act_title_2?smid=A2NXNERE0GDZFW&psc=1

In [ ]:
Choco123$$

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# Setup Selenium WebDriver
driver_path = r"C:\Users\New\Downloads\chromedriver-win32\chromedriver-win32\chromedriver.exe"
service = Service(driver_path)
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")
driver = webdriver.Chrome(service=service, options=options)

# Get the product URL from the user
product_url = input("Enter the Amazon product URL: ").strip()

# Navigate to the product URL
driver.get(product_url)
time.sleep(30)  # Wait for the page to load

# Extract product name
try:
    product_name = driver.find_element(By.ID, "productTitle").text.strip()
    print(f"Product Name: {product_name}")
except Exception as e:
    print(f"Error extracting product name: {e}")
    product_name = "Amazon_Product"

# Click on "See all reviews" if available
try:
    see_all_reviews = driver.find_element(By.XPATH, '//a[contains(text(), "See all reviews")]')
    see_all_reviews.click()
    time.sleep(5)
except Exception as e:
    print(f"Error clicking 'See all reviews': {e}")

# Initialize list to store review data
reviews_data = []

# Scrape all reviews
while True:
    try:
        # Wait for reviews to load
        time.sleep(3)

        # Extract reviews on the current page
        reviews = driver.find_elements(By.XPATH, '//div[@data-hook="review"]')
        print(f"Found {len(reviews)} reviews on this page.")

        for review in reviews:
            try:
                review_title = review.find_element(By.XPATH, './/a[@data-hook="review-title"]').text.strip()
            except:
                review_title = "No Title"

            try:
                review_body = review.find_element(By.XPATH, './/span[@data-hook="review-body"]').text.strip()
            except:
                review_body = "No Review Text"

            try:
                review_rating = review.find_element(By.XPATH, './/i[@data-hook="review-star-rating"]').text.strip()
            except:
                review_rating = "No Rating"

            try:
                review_date = review.find_element(By.XPATH, './/span[@data-hook="review-date"]').text.strip()
            except:
                review_date = "No Date"

            reviews_data.append({
                "Title": review_title,
                "Review": review_body,
                "Rating": review_rating,
                "Date": review_date,
            })

        # Check if "Next" button is available
        try:
            next_button = driver.find_element(By.XPATH, '//ul[@class="a-pagination"]//li[@class="a-last"]/a')
            next_button.click()
            time.sleep(5)
        except Exception as e:
            print("No more pages or error navigating to the next page.")
            break
    except Exception as e:
        print(f"Error extracting reviews: {e}")
        break

# Close the browser
driver.quit()

# Save to Excel
if reviews_data:
    output_file = f"{product_name}.xlsx"
    df = pd.DataFrame(reviews_data)
    df.to_excel(output_file, index=False)
    print(f"Reviews saved to {output_file}")
else:
    print("No reviews found.")


Enter the Amazon product URL:  https://www.amazon.in/gp/product/B0BGQMGNCW/ref=ox_sc_act_title_2?smid=A2NXNERE0GDZFW&psc=1


Product Name: Monthly Habit Tracker A4 Size with Goals for Self-improvement and Building Habits (Pack of 24 Sheets)
Error clicking 'See all reviews': Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(text(), "See all reviews")]"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00345093+25075]
	(No symbol) [0x002CE124]
	(No symbol) [0x001ABE63]
	(No symbol) [0x001EFD06]
	(No symbol) [0x001EFF4B]
	(No symbol) [0x0022D8C2]
	(No symbol) [0x00211EC4]
	(No symbol) [0x0022B48E]
	(No symbol) [0x00211C16]
	(No symbol) [0x001E3F3C]
	(No symbol) [0x001E4ECD]
	GetHandleVerifier [0x00632523+3094147]
	GetHandleVerifier [0x00645754+3172532]
	GetHandleVerifier [0x0063DF32+3141778]
	GetHandleVerifier [0x003E2100+668256]
	(No symbol) [0x002D6C4D]
	(No symbol) [0x002D3DF8]
	(No symbol) [0x00